In [ ]:
!pip3 install opencv-python
!pip3 install mediapipe
!pip3 install Pillow


In [1]:
import mediapipe as mp
import cv2
import numpy as np
import time
import uuid
import os
# import dlib
from PIL import Image
from skimage import io
import matplotlib.pyplot as plt
from keras.models import load_model
pred_model = load_model("./NSL.h5")

2022-05-02 20:34:38.877421: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sulav/.local/lib/python3.10/site-packages/cv2/../../lib64:
2022-05-02 20:34:38.877439: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-05-02 20:34:40.588394: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sulav/.local/lib/python3.10/site-packages/cv2/../../lib64:
2022-05-02 20:34:40.588414: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-02 20:34:40.588427: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver 

In [2]:
Categories = ['क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'श', 'ष', 'स', 'ह', 'क्ष', 'त्र', 'ज्ञ','del','nothing','space']

In [3]:


mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands



In [4]:
mp_drawing.DrawingSpec??

Init signature:
mp_drawing.DrawingSpec(
    color: Tuple[int, int, int] = (224, 224, 224),
    thickness: int = 2,
    circle_radius: int = 2,
) -> None
Docstring:      DrawingSpec(color: Tuple[int, int, int] = (224, 224, 224), thickness: int = 2, circle_radius: int = 2)
Source:        
@dataclasses.dataclass
class DrawingSpec:
  # Color for drawing the annotation. Default to the white color.
  color: Tuple[int, int, int] = WHITE_COLOR
  # Thickness for drawing the annotation. Default to 2 pixels.
  thickness: int = 2
  # Circle radius. Default to 2 pixels.
  circle_radius: int = 2
File:           ~/.local/lib/python3.10/site-packages/mediapipe/python/solutions/drawing_utils.py
Type:           type
Subclasses:     


In [6]:
import cv2
import mediapipe as mp

mphands = mp.solutions.hands
hands = mphands.Hands()
mp_drawing = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)

_, frame = cap.read()

h, w, c = frame.shape

while True:
    _, frame = cap.read()
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(framergb)
    hand_landmarks = result.multi_hand_landmarks
    if hand_landmarks:
        for handLMs in hand_landmarks:
            x_max = 0
            y_max = 0
            x_min = w
            y_min = h
            for lm in handLMs.landmark:
                x, y = int(lm.x * w), int(lm.y * h)
                if x > x_max:
                    x_max = x
                if x < x_min:
                    x_min = x
                if y > y_max:
                    y_max = y
                if y < y_min:
                    y_min = y
            cv2.rectangle(frame, (x_min-50, y_min-50), (x_max+50, y_max+50), (0, 255, 0), 2)
            # time.sleep(1)
            try:
                hand_frame = [x_min-50,y_min-50,x_max+50,y_max+50]
                framebgr = cv2.cvtColor(framergb, cv2.COLOR_RGB2BGR)
                # time.sleep(1)
                cropped_image = framebgr[hand_frame[1]:hand_frame[3],hand_frame[0]:hand_frame[2]]
                # time.sleep(1)
                # cv2.imwrite("./test.jpg",cropped_image)
                # plt.imshow(cropped_image)
                # plt.show()
                a = cv2.resize(cropped_image,(256,256)) 
                # time.sleep(1)
                a = a.reshape(1,256,256,3)
                # mp_drawing.draw_landmarks(frame, handLMs)
                # time.sleep(1)
                cv2.putText(frame,str(Categories[np.argmax(pred_model.predict(a))]),(x_max+5,y_max+5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            except:
                continue

    cv2.imshow("Frame", frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
         
        # print(pred_model.predict(a))
        
        cv2.destroyAllWindows()

        break

    cv2.waitKey(1)
cap.release()


In [3]:
test_image= cv2.imread("./apple1.jpg")
# test_image= cv2.imread("./apple.jpg")
aa = cv2.resize(test_image,(256,256))
aa = aa.reshape((1,256,256,3))
np.argmax(pred_model.predict(aa))

28

In [7]:
# image= cv2.imread('./images.jpeg')

# cropped= image[30:550, 100:530]
# cv2.imwrite("./test.jpg",cropped)

# # cv2.imshow('Cropped Image', image)